In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from torch.autograd import Variable
from torch import Tensor, optim, nn
import wandb
from tqdm import tqdm

wandb.login()

wandb: Currently logged in as: moritz-palm. Use `wandb login --relogin` to force relogin


True

In [2]:
config = {
    "learning_rate": 0.02,
    "architecture": "GRU",
    "dataset": "timeline_1.0",
    "epochs": 100,
    "classes": 2,
    "batch_size": 64,
    "num_layers": 2,
    "hidden_size": 64,
    "dropout_prob": 0.2,
    "input_size": 229,
    "output_size": 2,
    "optimizer": "Adam",
    "loss": "CrossEntropyLoss",
    "activation": "ReLU",
    "initializer": "Xavier",
    "regularization": "L2",
    "regularization_lambda": 0.01,
}

In [3]:
def model_pipeline(hyperparameters):
    with wandb.init(project="leaguify", config=hyperparameters):
        # access all HPs through wandb.config, so logging matches execution!
        config = wandb.config

        # make the model, data, and optimization problem
        model, train_loader, test_loader, criterion, optimizer = make(config)
        print(model)

        # and use them to train the model
        train(model, train_loader, criterion, optimizer, config)

        # and test its final performance
        test(model, test_loader)

    return model

In [4]:
class StaticDataset(Dataset):
    def __init__(self, data_dir, transform=None, target_transform=None):
        self.data = torch.tensor(np.load(data_dir)[:, :-1], dtype=torch.float32, device=device)
        self.labels = torch.tensor(np.load(data_dir)[:, -1], dtype=torch.int64, device=device)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx, 1:]
        label = self.labels[idx]
        if self.transform:
            sample = self.transform(sample)
        if self.target_transform:
            label = self.target_transform(label)
        return sample, label

In [5]:
def make(config):
    train, test = get_data(train=True), get_data(train=False)
    train_loader = make_loader(train, batch_size=config.batch_size)
    test_loader = make_loader(test, batch_size=config.batch_size)

    model = GRU(config.input_size, config.hidden_size, config.num_layers, config.dropout_prob,
                          config.classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

    return model, train_loader, test_loader, criterion, optimizer

In [6]:
def get_data(slice=1, train=True):
    if train:
        full_dataset = StaticDataset('../data/processed/train_timeline.npy')
    else:
        full_dataset = StaticDataset('../data/processed/test_timeline.npy')
    sub_dataset = torch.utils.data.Subset(full_dataset, range(0, len(full_dataset), slice))
    return sub_dataset

In [7]:
def make_loader(dataset, batch_size=64):
    return DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)

In [8]:
device = (
    "cuda" if torch.cuda.is_available()
    else "cpu"
)
if torch.cuda.is_available():
    print(f'PyTorch version: {torch.__version__}')
    print('*' * 10)
    print(f'_CUDA version: ')
    !nvcc --version
    print('*' * 10)
    print(f'CUDNN version: {torch.backends.cudnn.version()}')
    print(f'Available GPU devices: {torch.cuda.device_count()}')
    print(f'Device Name: {torch.cuda.get_device_name()}')
print(f"Using {device} device")

Using cpu device


In [9]:
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob = 0.2):
        super(GRU, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        return hidden

In [10]:
def train(model, loader, criterion, optimizer, config):
    wandb.watch(model, criterion, log='all', log_freq=10)

    total_batches = len(loader) * config.epochs
    example_count = 0
    batch_count = 0
    for epoch in tqdm(range(config.epochs)):
        for _, (matches, labels) in enumerate(loader):
            loss = train_batch(matches, labels, model, optimizer, criterion)
            example_count += len(matches)
            batch_count +=1
            if (batch_count + 1) % 25 == 0:
                train_log(loss, example_count, epoch)

In [11]:
def train_batch(matches, labels, model, optimizer, criterion):
    output = model(matches)
    loss = criterion(output, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss

In [12]:
def train_log(loss, example_count, epoch):
    wandb.log({"epoch": epoch, "loss": loss}, step=example_count)
    print(f"Loss after {str(example_count).zfill(5)} examples: {loss:.3f}")

In [13]:
def test(model, test_loader):
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        correct, total = 0, 0
        for matches, labels in test_loader:
            matches, labels = matches.to(device), labels.to(device)
            outputs = model(matches)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f"Accuracy of the model on the {total} " +
              f"test matches: {correct / total:%}")

        wandb.log({"test_accuracy": correct / total})

In [14]:
model = model_pipeline(config)

Traceback (most recent call last):
  File "/var/folders/ww/zccj65710l313xpntj6vpnth0000gn/T/ipykernel_87678/31012391.py", line 7, in model_pipeline
    model, train_loader, test_loader, criterion, optimizer = make(config)
  File "/var/folders/ww/zccj65710l313xpntj6vpnth0000gn/T/ipykernel_87678/1853932103.py", line 2, in make
    train, test = get_data(train=True), get_data(train=False)
  File "/var/folders/ww/zccj65710l313xpntj6vpnth0000gn/T/ipykernel_87678/2199354146.py", line 3, in get_data
    full_dataset = StaticDataset('../data/processed/train_timeline.npy')
  File "/var/folders/ww/zccj65710l313xpntj6vpnth0000gn/T/ipykernel_87678/1704340242.py", line 3, in __init__
    self.data = torch.tensor(np.load(data_dir)[:, :-1], dtype=torch.float32, device=device)
  File "/Users/admin/opt/miniconda3/envs/Leaguify/lib/python3.10/site-packages/numpy/lib/npyio.py", line 427, in load
    fid = stack.enter_context(open(os_fspath(file), "rb"))
FileNotFoundError: [Errno 2] No such file or direct

FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/train_timeline.npy'